# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [20]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [21]:
# this should Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [22]:
# this should Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
`SELECT 
      artist_name, song_title, song_length
  FROM 
      artist_playlist_in_session 
  WHERE  
      session_id = 338 AND item_in_session = 4`


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
`    SELECT
        artist_name, song_title, user_firstname, user_lastname
    FROM 
        playlist_session
    WHERE 
        session_id = 182 AND user_id = 10
    ORDER BY
        item_in_session`

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`SELECT
        user_firstname, user_lastname
    FROM 
        users_for_song
    WHERE 
        song_title = 'All Hands Against His Own' `


Create table for Query 1

In [23]:
## creating a table that handle this specific type of query (the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4)

query = "CREATE TABLE IF NOT EXISTS artist_playlist_in_session"
query += '''(
                session_id int,
                item_in_session int,
                song_title text,
                song_length float,
                artist_name text,
                PRIMARY KEY (session_id, item_in_session)
                )'''
# PRIMARY KEY:combinion of session_id, item_in_session and this means that the partition key is session_id, and the clustering key is item_in_session.
# which makes It would make sense that in a collection of songs you would want to store them by item_in_session.
                    
try:
    session.execute(query)
except Exception as e:
    print(e)

Insert into table for Query 1

In [24]:
# Loading data into table from csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO artist_playlist_in_session (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s) "
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        session.execute(query,(int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
    
#         session.execute(query, (dtype(line[col])
#                                for col, dtype in
#                                zip([8, 3, 0, 9, 5],[int, int, str, str, float]
#                                   )
#                                )
#                        )
#         print(line)
#         break

#### Do a SELECT to verify that the data have been inserted into each table

Select Query 1 from the table

In [25]:
## Validate the Data Model of Query 1

query = ("""
    SELECT
        artist_name, song_title, song_length
    FROM 
        artist_playlist_in_session
    WHERE 
        session_id = 338 AND item_in_session = 4
""")
rows = session.execute(query)
for row in rows:
    print("Artist:",row.artist_name,", Song Title:", row.song_title,", Song Length:", row.song_length)

Artist: Faithless , Song Title: Music Matters (Mark Knight Dub) , Song Length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Create table for Query 2

In [26]:
## creating a table that handle this specific query (Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182)

query = "CREATE TABLE IF NOT EXISTS playlist_session"
query += '''(
                artist_name text,
                song_title text,
                user_firstname text,
                user_lastname text,
                user_id int,
                session_id int,
                item_in_session int,
                PRIMARY KEY ((user_id, session_id),item_in_session)
                )'''
# here the PRIMARY KEY((user_id, session_id),item_in_session) this means that the composit partition key are user_id and session_id, and the clustering key is item_in_session.
                    
try:
    session.execute(query)
except Exception as e:
    print(e)  

Insert into table for Query 2

In [27]:
# Loading data into table from csv file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  INSERT statements into playlist_session table 
        query = "INSERT INTO playlist_session (artist_name, song_title, user_firstname, user_lastname, user_id, session_id, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        ## Assigning column element that should be assigned for each column in the INSERT statement.

        session.execute(query,(line[0], line[9], line[1], line[4], int(line[10]), int(line[8]), int(line[3])))


Select Query 2 from the table

In [28]:
## Validate the Data Model of Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

query = ("""
    SELECT
        artist_name, song_title, user_firstname, user_lastname
    FROM 
        playlist_session
    WHERE 
        session_id = 182 AND user_id = 10
        
""")
rows = session.execute(query)
for row in rows:
    print("Artist:",row.artist_name,", Song Title:", row.song_title,", User Name:", row.user_firstname, row.user_lastname)

Artist: Down To The Bone , Song Title: Keep On Keepin' On , User Name: Sylvie Cruz
Artist: Three Drives , Song Title: Greece 2000 , User Name: Sylvie Cruz
Artist: Sebastien Tellier , Song Title: Kilometer , User Name: Sylvie Cruz
Artist: Lonnie Gordon , Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , User Name: Sylvie Cruz


Create table for Query 3

In [14]:
## creating a table that handle this specific query ( every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own')

query = "CREATE TABLE IF NOT EXISTS users_for_song"
query += '''(
                song_title text,
                user_id int,
                user_firstname text,
                user_lastname text,
                PRIMARY KEY (song_title, user_id)
                )'''
# here the PRIMARY KEY(song_title, user_id) this means that the composit partition key is song_title, and the clustering key is user_id.

                    
try:
    session.execute(query)
except Exception as e:
    print(e)         

Insert into table for Query 3

In [15]:

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  INSERT statements into users_for_song table 
        query = "INSERT INTO users_for_song (song_title, user_firstname, user_lastname, user_id)"
        query = query + " VALUES (%s, %s, %s, %s) "
        ## Assign column element assigned for each column in the INSERT statement.

        session.execute(query,(line[9], line[1], line[4], int(line[10])))

Select Query 3 from the table

In [16]:
# TO-DO: Add in the SELECT statement to verify the data was entered into the table
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = ("""
    SELECT
         user_firstname, user_lastname
    FROM 
        users_for_song
    WHERE 
        song_title = 'All Hands Against His Own' 
        
""")
rows = session.execute(query)
for row in rows:
    print("User Name:", row.user_firstname, row.user_lastname)

User Name: Jacqueline Lynch
User Name: Tegan Levine
User Name: Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
## TO-DO: Drop the table before closing out the sessions
query = "drop table artist_playlist_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table users_for_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)        

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()